<a href="https://colab.research.google.com/github/ProfessorDong/Deep-Learning-Course-Examples/blob/master/RNN_Examples/CharacterGeneratorRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Natural Language Processing (Character Generation) with Recurrent Neural Network


We will generate charater using a RNN.  We will work with a dataset of Shakespeare's writing.  Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

- The model is character-based.
- The model is trained on small batches of text (100 characters each), and is able to generate a longer sequence of text with coherent structure.

*This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*


In [3]:
# %tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

###Download the Shakespeare Dataset

Here, we use an extract from a Shadespheare play for training.  We can use our own text paragraph data and use it for network training.

In [4]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# If you load your own data file, use the following lines:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

1130496/1115394 [==============================] - 0s 0us/step


###Read Contents of File
Let's look at the contents of the file.

In [5]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


###Encode
Encode each unique character as a different integer.

In [10]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [11]:
# Chect how a part of the text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


Make a function that can convert the numeric values back to text.


In [12]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Create Training Examples

Given a character, or a sequence of characters, what is the most probable next character? This is the task we are training the model to perform.

Our task is to feed the model a sequence and have it return the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

In [23]:
# Create a stream of characters from our text data
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Use the batch method to turn this stream of characters into batches of desired length
seq_length = 100  # length of sequence for a training example
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
# Use these sequences of length 101 and split them into input and output.
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [26]:
for x, y in dataset.take(3):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


EXAMPLE

INPUT
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us k

OUTPUT
ow Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki


Make training batches.

In [27]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Build the Model
We use an embedding layer, a LSTM layer, and one dense layer that contains a node for each unique character in our training data.  The dense layer will give us a probability distribution over all nodes.

In [28]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


###Create a Loss Function
Our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

We will create the loss function.

Before we do that, let's have a look at a sample input and the output from the untrained model. 

In [29]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [30]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-2.80563626e-03 -2.01189541e-03 -2.00248533e-03 ...  6.76438969e-04
    3.08970502e-03  3.34787741e-03]
  [-2.94530485e-03 -4.10257094e-03 -5.57637401e-03 ... -3.14545888e-03
    2.77015660e-03  5.63571416e-03]
  [-2.59278901e-03 -1.41776667e-03 -6.06804946e-03 ... -1.35958893e-03
    9.35940258e-03 -1.64926401e-04]
  ...
  [ 1.71243888e-03 -9.60265938e-03 -3.48372082e-03 ...  5.61608095e-03
   -2.61700037e-03 -2.35837977e-03]
  [ 8.13705847e-03 -1.35474969e-02  9.45026521e-04 ...  5.00989705e-03
    4.11230838e-03 -4.86602681e-03]
  [ 3.53081198e-03 -1.30859371e-02 -3.15532018e-03 ...  1.04979095e-04
    4.49208915e-03  7.20037206e-04]]

 [[-1.16737909e-03 -1.30258210e-03 -4.78682481e-03 ... -2.80769728e-03
   -1.16734882e-03 -3.45804961e-03]
  [-2.96951854e-03 -3.93426418e-03 -7.78678013e-03 ... -6.28750911e-03
    7.41826603e-04  1.72487868e-03]
  [ 6.59714395e-04  7.48090504e-04 -1.07238432e-02 ... -4.22818074e-03
    1.34937197e-03  2.76037026e-03]
  ...
  [ 7.998

In [31]:
# Let's examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00280564 -0.0020119  -0.00200249 ...  0.00067644  0.00308971
   0.00334788]
 [-0.0029453  -0.00410257 -0.00557637 ... -0.00314546  0.00277016
   0.00563571]
 [-0.00259279 -0.00141777 -0.00606805 ... -0.00135959  0.0093594
  -0.00016493]
 ...
 [ 0.00171244 -0.00960266 -0.00348372 ...  0.00561608 -0.002617
  -0.00235838]
 [ 0.00813706 -0.0135475   0.00094503 ...  0.0050099   0.00411231
  -0.00486603]
 [ 0.00353081 -0.01308594 -0.00315532 ...  0.00010498  0.00449209
   0.00072004]], shape=(100, 65), dtype=float32)


In [33]:
# and finally we look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-2.8056363e-03 -2.0118954e-03 -2.0024853e-03 -3.3417302e-03
 -4.8811152e-03  3.1842937e-04  2.2833475e-03 -5.9769084e-03
 -3.2897114e-03 -6.6977371e-03  3.4895211e-03 -1.0482825e-04
 -2.7434735e-03  1.1055754e-03  1.1608383e-03  5.9313366e-05
  2.4176119e-03  1.0602865e-03  1.0286545e-03 -6.0198721e-03
  1.7703356e-03 -3.9548301e-03  1.5860365e-03 -1.0261441e-03
  2.4976477e-03 -3.5006364e-03 -2.6917998e-03  6.7605237e-05
  2.3807299e-03 -5.2696844e-03  5.6201086e-04 -3.9872449e-04
  2.6879774e-04 -3.2235605e-03 -5.6983363e-03  3.1049215e-04
  3.4611265e-03 -2.7414435e-03  7.8588901e-03  4.8258905e-03
  3.7119684e-03  2.3285029e-03 -5.8958580e-04  1.8593203e-03
  1.0841693e-03  1.8353800e-03 -6.9397496e-04  3.2002064e-03
 -2.4679054e-03 -5.5446289e-03 -4.2931759e-03  3.2594975e-04
  3.3599518e-03  5.6126369e-03 -2.7155809e-03 -1.8854170e-03
 -2.3637689e-03 -1.6167050e-03  2.9680424e-03  1.5241861e-03
 -9.1516570e-04 -1.5719355e-03  6.7643897e-04  3.0897050e-03
  3.347877

In [34]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"iZSnF;EqMK,ilAwYxYE,VmdlYUxsjjpruIbyF?!EcZbEMtoktgVpZT$z:f:uTUlndzAKivfzgDXcXOP-'-yY'RFlKCwzWT3gxM!L"

So, we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were. 

In [35]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compile the Model
We can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next. 


In [36]:
model.compile(optimizer='adam', loss=loss)

###Create Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###Train the Model
**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**

In [38]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 31s 157ms/step - loss: 2.7441
Epoch 2/50
172/172 [==============================] - 29s 159ms/step - loss: 2.0481
Epoch 3/50
172/172 [==============================] - 29s 160ms/step - loss: 1.7885
Epoch 4/50
172/172 [==============================] - 29s 161ms/step - loss: 1.6345
Epoch 5/50
172/172 [==============================] - 29s 160ms/step - loss: 1.5366
Epoch 6/50
172/172 [==============================] - 29s 160ms/step - loss: 1.4709
Epoch 7/50
172/172 [==============================] - 29s 160ms/step - loss: 1.4230
Epoch 8/50
172/172 [==============================] - 29s 161ms/step - loss: 1.3856
Epoch 9/50
172/172 [==============================] - 29s 161ms/step - loss: 1.3544
Epoch 10/50
172/172 [==============================] - 29s 160ms/step - loss: 1.3283
Epoch 11/50
172/172 [==============================] - 29s 161ms/step - loss: 1.3037
Epoch 12/50
172/172 [==============================] - 29s 161ms/step - lo

###Load the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one piece of text to the model and have it make a prediction.

In [39]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model finishes training, we can find the **lastest checkpoint** that stores the models weights using the following line.

In [40]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load **any checkpoint** we want by specifying the exact file to load.

In [ ]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

###Generate Text
Generate some text using any starting string we choose.

Generating text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.  Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

![alt text](https://www.tensorflow.org/text/tutorials/images/text_generation_sampling.png)


In [41]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [44]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: First Citizen: Before we proceed any further, hear me speak.  All: Speak, speak.  First Citizen: You  OUTPUT irst Citizen: Before we proceed any further, hear me speak.  All: Speak, speak.
First Citizen: Before we proceed any further, hear me speak.  All: Speak, speak.  First Citizen: You  OUTPUT irst Citizen: Before we proceed any further, hear me speak.  All: Speak, speak.JzgJJzzbmVzbGzXzVzz&bzjzQzzzQzzQzQzzzzJzQzzJzzJzzVzQzzJzzzbQzQzzJJzJzzJxzzbRzzbzbRzbbQzFz&zzJzbzQzzzzJzzzzzzJzQxz&NzzzGzVzzJzxzzzJzQzzzJzQzzxzzbbz&z&zz&QzzxbQzzVzHbQzbQzJzzQzzzzbQzQxQzzzzxzQz&JzzzzzxJzzbm&zzz$bQzzJz&zzQxzzzQz$bz&zJzQzVzJJzzJzJzzzzjJzzzQzzzzzzzJzQzzz$bzXzbzbXzbVzbzXzH$zzQzzzQzzzQzzQzQzzJzzJxzzzzMzQzJzzzJJzzQzzxzQzJxzGzzzJzJzzJJxzzJz&zzzzQzQzzzVzzQzzxzJzJxz&b$zQzzzJzzzJzzzzzJ&$z&zzzzJzzJzzzQzkzzzzzQzzzzbz&xNz&zzzVzzbDzzzzzJz$bzzzXzqzzzzzVz&zQzzJzzzzzzQzzqzzzzQzzJzHzQzzzzzJzzzzz&&zzQzzzzzJzzzbzzXzzQzzzVbzzqmbzzJJzzbzJzzzzzJzHzzJzzbzVzzzJzzzzJzzQzzzJzQzzJzzQzzJzzMbQczQxzz3zzQxzzM

##Sources

1. Chollet François. Deep Learning with Python. Manning Publications Co., 2018.
2. “Text Generation with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_generation.
3. “Understanding LSTM Networks.” Understanding LSTM Networks -- Colah's Blog, https://colah.github.io/posts/2015-08-Understanding-LSTMs/.